## Zadanie domowe: BBHE i DSIHE

W klasycznym wyrównywaniu histogramu HE  po wykonaniu operacji jasność obrazu ulega zmianie.
Dało się to zaobserwować podczas przeprowadzonych eksperymentów.
Jeśli nie to należy uruchomić skrypt z sekcji A i zwrócić na to uwagę.
Średnia jasność dąży do środkowego poziomu szarości.
Jest to wada i dlatego klasyczne HE ma ograniczone zastosowanie.

Powstało sporo metod, które eliminują to niekorzystne zjawisko.
Najprostsze z nich polegają na dekompozycji obrazu wejściowego na dwa podobrazy (wg. pewnego kryterium).
Następnie operacja HE wykonywana jest dla tych podobrazów.

Dwie znane z literatury metody to:
- Bi-Histogram Equalization
- DSIHE - Dualistic Sub-Image Histogram Equalization

W metodzie BBHE za kryterium podziału przyjmuje się średnią jasność w obrazie.
W DSIHE obraz dzieli się na dwa podobrazy o takiej samej liczbie pikseli (jaśniejszych i ciemniejszych).

W ramach zadania należy zaimplementować wybraną metodę: BBHE lub DSIHE (ew. obie).

1. Wczytaj obraz *jet.bmp* i wylicz jego histogram.
2. W kolejnym kroku należy wyznaczyć próg podziału obrazu na dwa podobrazy (*lm*).
3. Dla BBHE wyznacz średnią jasność obrazu. Dla DSIHE można wykorzystać histogram skumulowany.
Należy znaleźć poziom jasności który znajduje się "w połowie" histogramu skumulowanego.
W tym celu warto stworzyć tablicę, zawierającą moduł histogramu skumulowanego pomniejszonego o połowę liczby pikseli.
Następnie znaleźć minimum - `np.argmin`.
4. Dalej należy podzielić histogram oryginalnego obrazu na dwa histogramy *H1* i *H2*.
Dla każdego z nich wyliczyć histogram skumulowany ($C_1$ i $C_2$) i wykonać normalizację.
Normalizacja polega na podzieleniu każdego histogramu przez jego największy element.
5. Na podstawie histogramów skumulowanych należy stworzyć przekształcenie LUT.
Należy tak przeskalować $C_1$ i $C_2$, aby uzyskać jednorodne przekształcenie.
Tablicę $C_1$ wystarczy pomnożyć przez próg podziału.
Tablicę $C_2$ należy przeskalować do przedziału: $<lm+1; 255>$, gdzie $lm$ jest progiem podziału.<br>
$C_{1n} = (lm)*C1;$<br>
$C_{2n} = lm+1 + (255-lm-1)*C2;$<br>
Następnie dwie części tablicy przekodowań należy połączyć.
6. Ostatecznie należy wykonać operację LUT i wyświetlić wynik wyrównywania histogramu.
Porównaj wynik operacji BBHE lub DSIHE z klasycznym HE.

In [ ]:
import cv2
import os
import requests
from matplotlib import pyplot as plt
import numpy as np

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/03_Histogram/'

fileName = 'jet.bmp'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

In [ ]:
img = cv2.imread(fileName, cv2.IMREAD_GRAYSCALE)
plt.imshow(img, cmap='gray')
plt.title("Oryginalny obraz")
plt.show()

In [ ]:
histogram = cv2.calcHist([img], [0], None, [256], [0, 256])

# HBBe
lm_bbhe = int(np.mean(img))
print("BBHE - lm:", lm_bbhe)

# DSIHE
cumulative_histogram = np.cumsum(histogram)
total_pixels = cumulative_histogram[-1]
half_pixels = total_pixels / 2

half_diff = np.abs(cumulative_histogram - half_pixels)
lm_dsihe = np.argmin(half_diff)
print("DSIHE - lm:", lm_dsihe)

In [ ]:
H1_bbhe = histogram[:lm_bbhe + 1]
H2_bbhe = histogram[lm_bbhe + 1:]

H1_dsihe = histogram[:lm_dsihe + 1]
H2_dsihe = histogram[lm_dsihe + 1:]

C1_bbhe = np.cumsum(H1_bbhe)
C2_bbhe = np.cumsum(H2_bbhe)

C1_dsihe = np.cumsum(H1_dsihe)
C2_dsihe = np.cumsum(H2_dsihe)

C1_bbhe = C1_bbhe / C1_bbhe.max()
C2_bbhe = C2_bbhe / C2_bbhe.max()

C1_dsihe = C1_dsihe / C1_dsihe.max()
C2_dsihe = C2_dsihe / C2_dsihe.max()

In [ ]:
# BBHE
C1n_bbhe = (lm_bbhe * C1_bbhe).astype(np.uint8)
C2n_bbhe = (lm_bbhe + 1 + (255 - lm_bbhe - 1) * C2_bbhe).astype(np.uint8)
LUT_bbhe = np.concatenate([C1n_bbhe, C2n_bbhe])

# DSIHE
C1n_dsihe = (lm_dsihe * C1_dsihe).astype(np.uint8)
C2n_dsihe = (lm_dsihe + 1 + (255 - lm_dsihe - 1) * C2_dsihe).astype(np.uint8)
LUT_dsihe = np.concatenate([C1n_dsihe, C2n_dsihe])

In [ ]:
img_bbhe = cv2.LUT(img, LUT_bbhe)
img_dsihe = cv2.LUT(img, LUT_dsihe)

img_he = cv2.equalizeHist(img)

plt.imshow(img_bbhe, cmap='gray')
plt.title("BBHE")
plt.show()

plt.imshow(img_dsihe, cmap='gray')
plt.title("DSIHE")
plt.show()

plt.imshow(img_he, cmap='gray')
plt.title("HE")
plt.show()

In [ ]:
hist_bbhe = cv2.calcHist([img_bbhe], [0], None, [256], [0, 256])
hist_dsihe = cv2.calcHist([img_dsihe], [0], None, [256], [0, 256])
hist_he = cv2.calcHist([img_he], [0], None, [256], [0, 256])

plt.plot(hist_bbhe, label="BBHE")
plt.plot(hist_dsihe, label="DSIHE")
plt.plot(hist_he, label="Klasyczne HE")
plt.legend()
plt.title("Porównanie histogramów")
plt.show()